In [ ]:
import glob
# from huggingface_hub import InferenceClient
import ollama
# from dotenv import load_dotenv
import os
import gradio as gr
# from IPython.display import display, Markdown

In [ ]:
# load_dotenv()
# hf_key = os.getenv('HF_KEY')
# MODEL = 'deepseek-ai/DeepSeek-V3.1'
# client = InferenceClient(
#     provider='novita',
#     api_key=hf_key
# )

In [ ]:
context = {}

employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
    # More robust way to extract name from file path
    name = os.path.splitext(os.path.basename(employee))[0].split(' ')[-1]
    try:
        with open(employee, 'r', encoding='utf-8') as f:
            doc = f.read()
        context[name] = doc
    except Exception as e:
        print(f"Error reading file {employee}: {e}")

In [ ]:
products = glob.glob("knowledge-base/products/*")

for product in products:
    # More robust way to extract name from file path
    name = os.path.splitext(os.path.basename(product))[0].split(' ')[-1]
    try:
        with open(product, 'r', encoding='utf-8') as f:
            doc = f.read()
        context[name] = doc
    except Exception as e:
        print(f"Error reading file {product}: {e}")

In [ ]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [ ]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)

    return relevant_context

In [ ]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [ ]:
# To use local ollama
ollama_model = 'gpt-oss:120b-cloud'

def chat(message, history):
    messages = [{'role': 'system', 'content': system_message}]
    for user_message, assisstant_message in history:
        messages.append({'role': 'user', 'content':user_message})
        messages.append({'role':'system', 'content': assisstant_message})

    message = add_context(message)
    messages.append({'role':'user', 'content': message})

    response = ollama.chat(model=ollama_model, messages=messages, stream=True)
    result = ''
    for chunk in response:
        result += chunk['message']['content']
        yield result

In [ ]:
gr.ChatInterface(fn = chat).launch()